<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"></ul></div>

In [35]:
import backtesting
from backtesting import Backtest, Strategy
from backtesting.lib import crossover
import sys
sys.path.append("..")
from utils import BBANDS_signal
from talib import BBANDS

gazp = pd.read_csv("../gazp.csv", parse_dates=["Date"]).sort_values("Date").set_index('Date')
gazp = gazp[:150]
# date = gazp['Date']
price = gazp["Close"].values

In [90]:
class BB(Strategy):
    std = 2
    loockback = 20

    def init(self):
        
        self.upper, self.middle, self.lower = self.I(
            BBANDS, self.data.Close, self.loockback, self.std, self.std, 0
        )

    def next(self):

        if crossover(self.data.Close, self.lower):
            print("long")
            self.buy()

        if self.position.is_long and (
            crossover(self.data.Close, self.middle)
            or crossover(self.lower, self.data.Close)
        ):
            print("close long")
            self.position.close()
            
        if crossover(self.upper, self.data.Close):
            print("short")
            self.sell()
            
        if self.position.is_short and (crossover(self.middle, self.data.Close) or crossover(self.upper, self.data.Close)):
            print("exit short")
            self.position.close()
            
bt = Backtest(gazp, BB, commission = .0006)
res = bt.run()

short
exit short
short
exit short
long
close long


/home/sergey/anaconda3/lib/python3.9/site-packages/backtesting/backtesting.py:1584: RuntimeWarning: divide by zero encountered in double_scalars
  s.loc['Sortino Ratio'] = np.clip(annualized_return / (np.sqrt(np.mean(day_returns.clip(-np.inf, 0)**2)) * np.sqrt(annual_trading_days)), 0, np.inf)  # noqa: E501


In [91]:
res._trades

,Size,EntryBar,ExitBar,EntryPrice,ExitPrice,PnL,ReturnPct,EntryTime,ExitTime,Duration
0,-44,71,90,222.766260,222.31,20.07544,0.002048,2006-02-01 16:25:00,2006-02-01 18:00:00,0 days 01:35:00
1,-44,100,105,223.365900,222.77,26.21960,0.002668,2006-02-02 10:35:00,2006-02-02 11:00:00,0 days 00:25:00
2,45,120,131,221.252672,221.75,22.37976,0.002248,2006-02-02 12:15:00,2006-02-02 13:10:00,0 days 00:55:00
